In [29]:
!pip install gspread
!pip install oauth2client

In [30]:
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [31]:
def auth():
    SP_CREDENTIAL_FILE='linebot.json'
    SP_SCOPE={
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
    }

    SP_SHEET_KEY='************'
    SP_SHEET='timeseet'

    credentials= ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE,SP_SCOPE)
    gc=gspread.authorize(credentials)

    worksheet=gc.open_by_key(SP_SHEET_KEY).worksheet(SP_SHEET)
    return worksheet

In [32]:
worksheet=auth()

In [33]:
worksheet

<Worksheet 'timeseet' id:0>

In [34]:
df=pd.DataFrame(worksheet.get_all_records())

In [35]:
timestamp=datetime.now()

In [36]:
date=timestamp.strftime('%Y/%m/%d')

In [37]:
punch_in=timestamp.strftime('%H:%M')

In [38]:
df=df.append({'日付':date,'出勤時間':punch_in,'退勤時間':'00:00'},ignore_index=True)

In [39]:
worksheet.update([df.columns.values.tolist()]+df.values.tolist())

{'spreadsheetId': '*******************',
 'updatedRange': 'timeseet!A1:C12',
 'updatedRows': 12,
 'updatedColumns': 3,
 'updatedCells': 36}

In [40]:
[df.columns.values.tolist()]+df.values.tolist()

[['日付', '出勤時間', '退勤時間'],
 ['2021/04/17', '10:00', '18:00'],
 ['2021/04/19', '22:10', '22:17'],
 ['2021/04/19', '22:42', '22:43'],
 ['2021/04/26', '21:21', '21:21'],
 ['2021/04/26', '21:21', '21:21'],
 ['2021/04/27', '20:31', '20:31'],
 ['2021/04/27', '20:31', '20:31'],
 ['2021/04/28', '21:47', '21:47'],
 ['2021/04/28', '21:47', '21:47'],
 ['2021/05/01', '23:08', '23:08'],
 ['2021/05/02', '18:09', '00:00']]

In [41]:
df.values.tolist()

[['2021/04/17', '10:00', '18:00'],
 ['2021/04/19', '22:10', '22:17'],
 ['2021/04/19', '22:42', '22:43'],
 ['2021/04/26', '21:21', '21:21'],
 ['2021/04/26', '21:21', '21:21'],
 ['2021/04/27', '20:31', '20:31'],
 ['2021/04/27', '20:31', '20:31'],
 ['2021/04/28', '21:47', '21:47'],
 ['2021/04/28', '21:47', '21:47'],
 ['2021/05/01', '23:08', '23:08'],
 ['2021/05/02', '18:09', '00:00']]

In [42]:
timestamp=datetime.now()

In [43]:
punch_out=timestamp.strftime('%H:%M')
punch_out

'18:09'

In [44]:
df.iloc[-1,2]=punch_out

In [45]:
worksheet.update([df.columns.values.tolist()]+df.values.tolist())

{'spreadsheetId': '*******************',
 'updatedRange': 'timeseet!A1:C12',
 'updatedRows': 12,
 'updatedColumns': 3,
 'updatedCells': 36}

In [46]:
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def auth():
    SP_CREDENTIAL_FILE='linebot.json'
    SP_SCOPE={
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
    }

    SP_SHEET_KEY='***********************'
    SP_SHEET='timeseet'

    credentials= ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE,SP_SCOPE)
    gc=gspread.authorize(credentials)

    worksheet=gc.open_by_key(SP_SHEET_KEY).worksheet(SP_SHEET)
    return worksheet

#出勤時間
def punch_in():
    worksheet=auth()
    df=pd.DataFrame(worksheet.get_all_records())

    timestamp=datetime.now()
    date=timestamp.strftime('%Y/%m/%d')
    punch_in=timestamp.strftime('%H:%M')

    df=df.append({'日付':date,'出勤時間':punch_in,'退勤時間':'00:00'},ignore_index=True)
    worksheet.update([df.columns.values.tolist()]+df.values.tolist())
    print('勤怠(出勤)登録完了しました')
#退勤時間
def punch_out():
    worksheet=auth()
    df=pd.DataFrame(worksheet.get_all_records())

    timestamp=datetime.now()
    punch_out=timestamp.strftime('%H:%M')

    df.iloc[-1,2]=punch_out
    worksheet.update([df.columns.values.tolist()]+df.values.tolist())
    print('勤怠(退勤)登録完了しました')

In [47]:
punch_in()

勤怠(出勤)登録完了しました


In [48]:
punch_out()

勤怠(退勤)登録完了しました


In [49]:
!pip3 install flask
!pip3 install line-bot-sdk

In [50]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

In [51]:
app = Flask(__name__)

YOUR_CHANNEL_ACCESS_TOKEN='**********************='
YOUR_CHANNEL_SECRET='******************'

line_bot_api = LineBotApi(YOUR_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(YOUR_CHANNEL_SECRET)
@app.route("/")

def hello_world():
    return "hello world"

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))


if __name__ == "__main__":
    app.run()